<a href="https://colab.research.google.com/github/LaugeSolvang/CarRental/blob/master/PreProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries

In [24]:
from google.colab import userdata, files
import os
import pandas as pd
import numpy as np

Acces github

In [25]:
os.environ['GITHUB_TOKEN'] = userdata.get('GITHUB_TOKEN')
%cd /content
!git clone https://$GITHUB_TOKEN@github.com/MatRitchie/Sunshine.git

%cd /content/Sunshine
!git pull https://$GITHUB_TOKEN@github.com/MatRitchie/Sunshine.git main

/content
fatal: destination path 'Sunshine' already exists and is not an empty directory.
/content/Sunshine
From https://github.com/MatRitchie/Sunshine
 * branch            main       -> FETCH_HEAD
Already up to date.


In [26]:
FILES = {
    "evolved": "incubator_project_graduated_from_graduation.csv",
    "retired": "incubator_project_metrics_graduated_retired.csv",
    "bypassed": "non_incubator_project_metrics.csv"
}
OUTPUT_FILE = "combined_project_status.csv"

LABELS = [
    'project', 'COM-1', 'COM-3', 'POP-4', 'STA-1', 'STA-2', 'STA-4', 'STA-5',
    'STA-6', 'STA-7', 'STA-10', 'STA-11', 'STA-12', 'TEC-1', 'TEC-2.1',
    'TEC-2.2', 'TEC-2.3', 'TEC-2.4', 'TEC-4', 'TEC-5', 'SWQ-1', 'SWQ-4.1',
    'SWQ-4.2', 'SWQ-4.3', 'SWQ-4.4', 'SWQ-4.5', 'SWQ-4.6', 'SWQ-4.7',
    'init', 'end', 'frequency (weeks)', 'project_url'
]

##Load and process the files

In [27]:
dfs = {}
for status, filepath in FILES.items():
    try:
        df = pd.read_csv(filepath)
        df.columns = LABELS[:min(len(df.columns), len(LABELS))]

        df.rename(columns={
            'COM-1'   : 'COM-1',
            'COM-3'   : 'COM-2',
            'POP-4'   : 'POP-1',
            'STA-1'   : 'STA-1',
            'STA-2'   : 'STA-2',
            'STA-4'   : 'STA-3',
            'STA-5'   : 'STA-4',
            'STA-6'   : 'STA-5',
            'STA-7'   : 'STA-6',
            'STA-10'  : 'STA-7',
            'STA-11'  : 'STA-8',
            'STA-12'  : 'STA-9',
            'TEC-1'   : 'TEC-1',
            'TEC-2.4' : 'TEC-2',
            'TEC-4'   : 'TEC-3',
            'TEC-5'   : 'TEC-4',
            'SWQ-1'   : 'SWQ-1',
            'SWQ-4.1' : 'SWQ-2.1',
            'SWQ-4.2' : 'SWQ-2.2',
            'SWQ-4.3' : 'SWQ-2.3',
            'SWQ-4.4' : 'SWQ-2.4',
            'SWQ-4.5' : 'SWQ-2.5',
            'SWQ-4.6' : 'SWQ-2.6',
            'SWQ-4.7' : 'SWQ-2.7',
        }, inplace=True)

        if 'TEC-2.2' in df.columns:
          df.drop(columns=['TEC-2.2'], inplace=True)

        if 'frequency (weeks)' in df.columns:
          df.drop(columns=['frequency (weeks)'], inplace=True)

        if 'STA-6' in df.columns:
          df['STA-6'] = df['STA-6'].apply(lambda x: 1 if str(x).lower() == 'true' else (0 if str(x).lower() == 'false' else x))

        # Calculate project age from STA-1 if present
        if 'STA-1' in df.columns:
            df['STA-1'] = (2025 - pd.to_datetime(df['STA-1'], errors='coerce').dt.year).fillna(-1).astype(int)

        df['status'] = status
        dfs[status] = df
    except Exception as e:
        print(f"Error processing {filepath}: {e}")

# Set project status based on presence in evolved list

In [28]:
if 'evolved' in dfs and 'retired' in dfs:
    evolved_projects = set(dfs['evolved']['project'])
    dfs['retired']['status'] = dfs['retired']['project'].apply(
        lambda proj: 'graduated' if proj in evolved_projects else 'retired'
    )

In [29]:
df_combined = pd.concat([df for df in dfs.values() if df is not None], ignore_index=True)

df_clean = df_combined.dropna(axis=1, how='all')
df_clean = df_clean[~df_clean.astype(str).apply(lambda x: x.str.contains('missing', case=False, na=False)).any(axis=1)]
df_clean = df_clean.dropna()

try:
    df_clean.to_csv(OUTPUT_FILE, index=False)
except Exception as e:
    print(f"Error saving output: {e}")

Pushing processed data

In [30]:
# Stage the new files for Git
!git add {OUTPUT_FILE}

# Commit the changes
!git commit -m "Add preprocessed data for combined files with removed missing data"

# Configure Git identity
!git config --global user.email "auto@example.com"  # Replace with your email
!git config --global user.name "Auto"        # Replace with your name

# Push the changes to GitHub
!git push https://$GITHUB_TOKEN@github.com/MatRitchie/Sunshine.git main

[main 42b0dbc] Add preprocessed data for combined files with removed missing data
 1 file changed, 526 insertions(+), 526 deletions(-)
Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 3.19 KiB | 1.06 MiB/s, done.
Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/MatRitchie/Sunshine.git
   9ab796b..42b0dbc  main -> main
